# Graphical Pipelines

### Content of this Notebook:
* Understanding what are graphical pipelines
* Understanding the API of graphical pipelines
* Examples of simple pipelines and how they can be implemented with graphical pipelines.
* More complex graphical pipeline
    * Forecasting
* Grid search with a graphical pipeline


The previously presented pipelines are sequential pipelines. I.e., the steps in the pipeline are sequentially ordered.

<img src="img/sequential_pipeline.png" width=750 />

Many tasks are non sequential. To solve this two possibilities exist:
1. Nesting Sequential Pipelines.
2. Using Graphical Pipelines.


Thus, there is the generalised graphial pipeline.
* Graphical means that different steps may share the same predecessor or provide their outputs to the same successor (the dataflows can branch and merge).
<img src="img/graphical_pipeline.png" width=750 />


* Generalised means that the pipeline can be used for multiple tasks (e.g. forecasting, classification, ...).

**Note**

The graphical pipeline is still experimental. Thus, this graphical should not used in production. However, we would be happy to get feedback on the graphical pipeline.



### Potential Use-Cases
There exist various potential use-case for the graphical pipeline. In the following, we focus on a forecasting and a classification pipeline.
#### Forecasting Use-Case for Graphical Pipelines


In forecasting tasks, the input of forecasters might depend on the output of other forecasters, which same the same input. I.e., there is a branching of the data flow since the same input is used for different forecasters and a merging of data flow since the forcasters' outputs are combined.
<img src="img/graphical_pipeline_example.png" width=750 />

This can be easily realised using the graphical pipeline.

#### Classification Use-Case for Graphical Pipelines


In classification taks, the input of classifier may rely on different features. Potentially, not all of these features are always observable. Thus, a soft sensor is required. Such a soft-sensor could be realised using a regressor.

<img src="img/graphical_pipeline_softsensor.png" width=750 />

For such a scenario, the graphical pipeline is a natural fit since it enables the combination of different tasks in one pipeline.
Note that in the current experimental state of the graphical pipeline, this use-case is not fully supported. However, we are working on this.

### Credits
The graphical pipeline was first developed by pyWATTS and was then adapted for sktime. The original implementation can be found [here](). pyWATTS is a open source library developed at the Institute of Applied Informatics and Automation at the KIT.


In [21]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import StandardScaler
from transformations.series.subset import ColumnSelect

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.datasets import load_arrow_head, load_longley, load_macroeconomic
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    ColumnEnsembleForecaster,
    ForecastX,
    MultiplexForecaster,
    make_reduction,
)
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
    temporal_train_test_split,
)
from sktime.forecasting.sarimax import SARIMAX
from sktime.performance_metrics.forecasting import mean_absolute_error
from sktime.pipeline.pipeline import Pipeline
from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.transformations.series.difference import Differencer
from sktime.transformations.series.exponent import ExponentTransformer

## How to build a Graphical Pipeline
Two ways to build a graphical pipeline:

1. Pass all steps to the pipeline during initialisation as for the sequential pipelines.

```python
pipeline = Pipeline([
    {"skobject": skobject1, "name": "bar", "edges": {"X": "y"}},
    {
        "skobject": skobject2,
        "name": "foo",
        "edges": {"X": "X", "y": "bar"},
    },
    {
        "skobject": skobject1,
        "name": "bar_inverse",
        "edges": {"X": "foo"},
        "method": "inverse_transform",
    },
])
```

2. Create a pipeline object and add the steps one by one.

```python

pipeline = Pipeline()
pipeline = pipeline.add_step(skobject1, "bar", edges={"X": "y"})
pipeline = pipeline.add_step(skobject2, "foo", edges={"X": "X", "y": "bar"})
pipeline = pipeline.add_step(
    skobject1, "bar_inverse", edges={"X": "foo"}, method="inverse_transform"
)
```

Thereby the `add_step` or the dicts in the step list during initialisation have the following parameters:

* skobject: The sktime object that should be added to the pipeline
* name: The name of the step
* edges: A dictionary that specifies the edges of the graph. The keys of the dictionary are the input arguments of the sktime object and the values are the names of the steps that should be connected to the input argument.
* method: The method of the sktime object that should be called. If no method is specified, the default method would be inferred based on the added skobject. This parameter is used for the inverse_transform method. Optional.
* kwargs: Additional keyword arguments that should be passed to the sktime object. Optional.

In the following, we show a few simple examples of the graphical pipeline, before we show more complex ones.

## Examples
### Forecasting Pipeline
In the following, we show how a simple forecasting pipeline could be implemented using the graphical pipeline. The pipeline consists of the following steps:


In [2]:
general_pipeline = Pipeline()
differencer = Differencer()

general_pipeline = general_pipeline.add_step(
    differencer, "differencer", edges={"X": "y"}
)
general_pipeline = general_pipeline.add_step(
    SARIMAX(), "sarimax", edges={"X": "X", "y": "differencer"}
)
general_pipeline = general_pipeline.add_step(
    differencer, "differencer_inv", edges={"X": "sarimax"}, method="inverse_transform"
)

The pipeline can be visualised as follows:
<img src="img/forecasting_pipeline.png" width=750 />



In [3]:
y, X = load_longley()
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X)

general_pipeline.fit(y=y_train, X=X_train, fh=[1, 2, 3, 4])
general_pipeline.predict(X=X_test)

1959    67213.735360
1960    68328.076304
1961    68737.861389
1962    71322.894013
Freq: A-DEC, Name: TOTEMP, dtype: float64

**Alternative Way in Defining the Pipeline**
An alternative to define a graphical pipeline would be to pass a list of steps to the Pipeline during creation. This would look as follows:

In [4]:
differencer = Differencer()

general_pipeline = Pipeline(
    [
        {"skobject": differencer, "name": "differencer", "edges": {"X": "y"}},
        {
            "skobject": SARIMAX(),
            "name": "sarimax",
            "edges": {"X": "X", "y": "differencer"},
        },
        {
            "skobject": differencer,
            "name": "differencer_inv",
            "edges": {"X": "sarimax"},
            "method": "inverse_transform",
        },
    ]
)

### Classification Pipeline
In the following, we show how a simple classification pipeline could be implemented using the graphical pipeline. The pipeline consists of the following steps:

In [5]:
general_pipeline = Pipeline()
general_pipeline = general_pipeline.add_step(
    ExponentTransformer(), "exponent", edges={"X": "X"}
)
general_pipeline = general_pipeline.add_step(
    KNeighborsTimeSeriesClassifier(), "classifier", edges={"X": "exponent", "y": "y"}
)

This pipeline can be visualised as follows:

<img src="img/classification_pipeline.png" width=750 />


In [6]:
X, y = load_arrow_head(split="train", return_X_y=True)
general_pipeline.fit(X=X, y=y)
general_pipeline.predict(X=X)

array(['0', '1', '2', '0', '1', '2', '0', '1', '2', '0', '1', '2', '0',
       '1', '2', '0', '1', '2', '0', '1', '2', '0', '1', '2', '0', '1',
       '2', '0', '1', '2', '0', '1', '2', '0', '1', '2'], dtype='<U1')

## A more Complex Example with Grid Search

The considered use-case is to forecast the inflation using forecasts of the real gross domestic product, real disposable personal income, and the unemployment rate. Furthermore the unemployment rate is forecasted using the same features except the unemployment rate itself.

<img src="img/graphical_pipeline_example.png" width=750 />


The data is taken from the macrodata dataset from the statsmodels package.


#### Pipeline Definition

In [7]:
pipeline = Pipeline()
sklearn_scaler = StandardScaler()
sktime_scaler = TabularToSeriesAdaptor(sklearn_scaler)
deseasonalizer = Deseasonalizer(sp=4)
detrender = Detrender()

pipeline = pipeline.add_step(
    sktime_scaler, name="scaler", edges={"X": "X__realgdp_realdpi_unemp"}
)
pipeline = pipeline.add_step(
    detrender, name="deseasonalizer", edges={"X": "X__realgdp_realdpi"}
)

pipeline = pipeline.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_gdp",
    edges={"y": "deseasonalizer__realgdp"},
)

pipeline = pipeline.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_dpi",
    edges={"y": "deseasonalizer__realdpi"},
)

pipeline = pipeline.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_unemp",
    edges={
        "y": "scaler__unemp",
        "X": [
            "forecaster_gdp",
            "forecaster_dpi",
        ],
    },
)

pipeline = pipeline.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_inflation",
    edges={"X": ["forecaster_dpi", "forecaster_unemp"], "y": "y"},
)

In [8]:
data = load_macroeconomic()

X = data[["realgdp", "realdpi", "unemp"]]
y = data[["infl"]]
fh = ForecastingHorizon([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

y_train, y_test, X_train, X_test = temporal_train_test_split(y, X=X, fh=fh)
X_train

,realgdp,realdpi,unemp
Period,,,
1959Q1,2710.349,1886.9,5.8
1959Q2,2778.801,1919.7,5.1
1959Q3,2775.488,1916.4,5.3
1959Q4,2785.204,1931.3,5.6
1960Q1,2847.699,1955.5,5.2
...,...,...,...
2005Q3,12683.153,9308.0,5.0
2005Q4,12748.699,9358.7,4.9
2006Q1,12915.938,9533.8,4.7


In [9]:
pipeline.fit(y=y_train, X=X_train, fh=fh)
result = pipeline.predict(X=None, fh=y_test.index)
((result - y_test) ** 2).mean()

infl    18.041837
dtype: float64

In [10]:
ridge = make_reduction(Ridge(), windows_identical=False, window_length=5)
ridge.fit(y=y_train, fh=fh)
((ridge.predict() - y_test) ** 2).mean()

infl    19.608558
dtype: float64

#### Grid Search

This pipeline has multiple parameters that might be tested to find the configurations. These parameters include:
* which forecaster should be used for which variable -> `MultiplexForecaster`
* what should be the hyperparameters of the forecaster
* which features should be used for the different forecasters -> Tune the edges of the graphical pipeline!

<img src="img/graphical_pipeline_example_grid.png" width=750 />

Since we do forecasting, we use the ForecastingGridSearchCV.

1. Specify the parameter grid:

The keys of the dictionary are the names of the steps in the pipeline and the values are the different configurations that should be tested for the step. Thus, to change the parameters of a skobject in the pipeline the key looks like: `step_name__skobject_name__parameter_name`. To change the inputs you need to vary the edges. This can be done with keys following the following scheme: `step_name_edges_Xory`


In [11]:
param_grid = {
    "forecaster_inflation__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_unemp__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_dpi__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_gdp__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_inflation__edges__X": [
        ["forecaster_unemp"],
        ["forecaster_unemp", "forecaster_dpi"],
    ],
    "forecaster_unemp__edges__X": [
        [],
        ["forecaster_dpi"],
        ["forecaster_gdp", "forecaster_dpi"],
    ],
}

Initialise the gridsearch using pipeline, crossvalidation strategy, scoringm and param_grid.


In [12]:
gridcv = ForecastingGridSearchCV(
    pipeline,
    cv=SlidingWindowSplitter(
        window_length=len(X_train) - 20,
        step_length=4,
        fh=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    ),
    scoring=mean_absolute_error,
    # refit=False,
    error_score="raise",
    param_grid=param_grid,
)

Call fit on the gridsearch object.

In [13]:
gridcv.fit(y=y_train, X=X_train)

C:\Users\bi4372\.conda\envs\pyWATTS-deRSE-2023\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.336e+03, tolerance: 2.727e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\bi4372\.conda\envs\pyWATTS-deRSE-2023\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.676e+03, tolerance: 2.996e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\bi4372\.conda\envs\pyWATTS-deRSE-2023\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or co

ForecastingGridSearchCV(cv=SlidingWindowSplitter(fh=[1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                     10, 11, 12],
                                                 step_length=4,
                                                 window_length=171),
                        error_score='raise',
                        forecaster=Pipeline(steps=[{'edges': {'X': 'X__realgdp_realdpi_unemp'},
                                                    'kwargs': {},
                                                    'method': None,
                                                    'name': 'scaler',
                                                    'skobject': TabularToSeriesAdaptor(transformer=StandardScaler())},
                                                   {'edges': {'X': 'X__realgdp_realdpi'},
                                                    'kwa...
                                    'forecaster_inflation__edges__X': [['forecaster_unemp'],
                                                                       ['forecaster_unemp',
                                                                        'forecaster_dpi']],
                                    'forecaster_inflation__skobject__selected_forecaster': ['ridge',
                                                                                            'lasso'],
                                    'forecaster_unemp__edges__X': [[],
                                                                   ['forecaster_dpi'],
                                                                   ['forecaster_gdp',
                                                                    'forecaster_dpi']],
                                    'forecaster_unemp__skobject__selected_forecaster': ['ridge',
                                                                                        'lasso']},
                        scoring=<function mean_absolute_error at 0x00000254DB71EC10>)

In [14]:
gridcv.cv_results_

,mean_test__DynamicForecastingErrorMetric,mean_fit_time,mean_pred_time,params,rank_test__DynamicForecastingErrorMetric
0,1.539329,0.098928,0.045676,{'forecaster_dpi__skobject__selected_forecaste...,54.5
1,1.720565,0.078510,0.041738,{'forecaster_dpi__skobject__selected_forecaste...,58.5
2,1.311031,0.150897,0.075369,{'forecaster_dpi__skobject__selected_forecaste...,1.5
3,3.116475,0.178225,0.083630,{'forecaster_dpi__skobject__selected_forecaste...,95.5
4,1.851588,0.103475,0.047466,{'forecaster_dpi__skobject__selected_forecaste...,65.0
...,...,...,...,...,...
91,1.443361,0.157408,0.077803,{'forecaster_dpi__skobject__selected_forecaste...,46.5
92,1.443361,0.169744,0.088619,{'forecaster_dpi__skobject__selected_forecaste...,46.5
93,1.443361,0.167966,0.083686,{'forecaster_dpi__skobject__selected_forecaste...,46.5
94,1.443361,0.191664,0.102800,{'forecaster_dpi__skobject__selected_forecaste...,46.5


In [15]:
result = gridcv.predict(X=None, fh=y_test.index)
((result - y_test) ** 2).mean()

infl    19.244087
dtype: float64

In [16]:
gridcv.best_params_

{'forecaster_dpi__skobject__selected_forecaster': 'ridge',
 'forecaster_gdp__skobject__selected_forecaster': 'ridge',
 'forecaster_inflation__edges__X': ['forecaster_unemp'],
 'forecaster_inflation__skobject__selected_forecaster': 'ridge',
 'forecaster_unemp__edges__X': ['forecaster_dpi'],
 'forecaster_unemp__skobject__selected_forecaster': 'ridge'}

In [17]:
gridcv.best_score_

1.3110312019415922

### How to implement the pipeline above by nesting sequential pipelines


In [18]:
data = load_macroeconomic()

X = data[["realgdp", "realdpi", "unemp"]]
y = data[["infl"]]
fh = ForecastingHorizon([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

y_train, y_test, X_train, X_test = temporal_train_test_split(y, X=X, fh=fh)

forecasting_pipeline_gdp = (
    ColumnSelect(["realgdp"])
    * Deseasonalizer()
    * MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    )
)
forecasting_pipeline_dpi = (
    ColumnSelect(["realdpi"])
    * Deseasonalizer()
    * MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    )
)

transform_unemp = ColumnSelect(["unemp"]) * TabularToSeriesAdaptor(StandardScaler())

input_unemp_forecast = ColumnSelect(["realgdp", "realdpi"]) * ColumnEnsembleForecaster(
    [
        ("realgdp", forecasting_pipeline_gdp, "realgdp"),
        ("realdpi", forecasting_pipeline_dpi, "realdpi"),
    ]
)

unemp_forecastt = transform_unemp * ForecastX(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    input_unemp_forecast,
)

input_inflation_forecast = ColumnSelect(
    ["realdpi", "unemp"]
) * ColumnEnsembleForecaster(
    [
        ("realdpi", forecasting_pipeline_dpi, "realdpi"),
        ("unemp", unemp_forecastt, "unemp"),
    ]
)
inflation_forecast = ForecastX(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    input_inflation_forecast,
)

In [19]:
inflation_forecast.fit(y=y_train, X=X_train, fh=fh)

ForecastX(forecaster_X=TransformedTargetForecaster(steps=[ColumnSelect(columns=['realdpi',
                                                                                'unemp']),
                                                          ColumnEnsembleForecaster(forecasters=[('realdpi',
                                                                                                 TransformedTargetForecaster(steps=[Deseasonalizer(),
                                                                                                                                    MultiplexForecaster(forecasters=[('ridge',
                                                                                                                                                                      RecursiveTabularRegressionForecaster(estimator=Ridge(),
                                                                                                                                                                                                           window_length=5)),
                                                                                                                                                                     ('lasso',
                                                                                                                                                                      RecursiveTabularRegressio...
                                                                                                                                                                                             RecursiveTabularRegressionForecaster(estimator=Ridge(),
                                                                                                                                                                                                                                  window_length=5)),
                                                                                                                                                                                            ('lasso',
                                                                                                                                                                                             RecursiveTabularRegressionForecaster(estimator=Lasso(),
                                                                                                                                                                                                                                  window_length=5))]))]),
                                                                                                 'unemp')])]),
          forecaster_y=MultiplexForecaster(forecasters=[('ridge',
                                                         RecursiveTabularRegressionForecaster(estimator=Ridge(),
                                                                                              window_length=5)),
                                                        ('lasso',
                                                         RecursiveTabularRegressionForecaster(estimator=Lasso(),
                                                                                              window_length=5))]))

In [20]:
# TODO inflation_forecast.predict() Currently is not working since
#  the shapes do not match...

ValueError: could not broadcast input array from shape (3,5) into shape (1,2,5)

In [ ]:
inflation_forecast.get_params(True)